In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [2]:

import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures 
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import StratifiedKFold, GridSearchCV 
from sklearn.metrics import accuracy_score

train_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv") 
test_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")

train_df.drop(columns=['Unnamed: 0'], inplace=True) 
test_df.drop(columns=['Unnamed: 0'], inplace=True)

X = train_df.drop(columns=['class', 'ID']) 
y = train_df['class'] 
X_test = test_df.drop(columns=['ID'])

y_encoded = LabelEncoder().fit_transform(y)


date_cols = X.columns.tolist()


for df in [X, X_test]: 
    df['ndvi_mean'] = df[date_cols].mean(axis=1) 
    df['ndvi_std'] = df[date_cols].std(axis=1) 
    df['ndvi_min'] = df[date_cols].min(axis=1) 
    df['ndvi_max'] = df[date_cols].max(axis=1) 
    df['ndvi_median'] = df[date_cols].median(axis=1) 
    df['ndvi_skew'] = df[date_cols].skew(axis=1) 
    df['ndvi_nan_count'] = df[date_cols].isna().sum(axis=1) 
    df['ndvi_above_03'] = (df[date_cols] > 0.3).sum(axis=1) 
    df['ndvi_below_01'] = (df[date_cols] < 0.1).sum(axis=1)
    diffs = df[date_cols].diff(axis=1).dropna(axis=1)
    df['ndvi_diff_mean'] = diffs.mean(axis=1)
    df['ndvi_diff_std'] = diffs.std(axis=1)

imputer = SimpleImputer(strategy='mean') 
X_imputed = imputer.fit_transform(X) 
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X_imputed) 
X_test_scaled = scaler.transform(X_test_imputed)


poly = PolynomialFeatures(degree=2, include_bias=False) 
X_poly = poly.fit_transform(X_scaled) 
X_test_poly = poly.transform(X_test_scaled)

param_grid = {'C': [0.1, 0.5, 1.0, 2.0, 5.0]} 
model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs') 
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) 
grid = GridSearchCV(model, param_grid, cv=skf, scoring='accuracy') 
grid.fit(X_poly, y_encoded)

print(f"Best Logistic Regression C: {grid.best_params_['C']}") 
print(f"Cross-validated Accuracy: {grid.best_score_:.5f}")


best_model = grid.best_estimator_ 
test_preds_encoded = best_model.predict(X_test_poly) 
label_encoder = LabelEncoder().fit(y) 
test_preds = label_encoder.inverse_transform(test_preds_encoded)

submission_df = pd.DataFrame({ 'ID': test_df['ID'], 'class': test_preds }) 
submission_df.to_csv("submission_logistic_regression.csv", index=False) 
print("Saved submission_logistic_regression.csv with maxed-out features.")



/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best Logistic Regression C: 0.1
Cross-validated Accuracy: 0.95000
Saved submission_logistic_regression.csv with maxed-out features.
